## Генерация диалога с помощью RuDlg-Mistral

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [ ]:
BASE_MODEL = "Open-Orca/Mistral-7B-OpenOrca"
ADAPTER_MODEL = "checkpoints/checkpoint-3900"
MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
RESPONSE_TEMPLATE = "<s>bot\n"
SYSTEM_PROMPT = """Ты – чат-бот технической поддержки Xiaomi Store, который помогает клиенту выбрать наиболее подходящий для него смартфон. Опираясь на описание смартфонов, помоги клиенту выбрать наиболее подходящий для него смартфон. Если ответа на вопрос клиента нет в приведенном описании, ответь "У меня не достаточно информации для ответа на ваш вопрос. Обратитесь пожалуйста к менеджеру в telegram".{personality}
Описание смартфонов из интернет-магазина сотовой связи Xiaomi Store:
1. Смартфон Xiaomi Redmi Note 10 Pro: Этот смартфон оснащен дисплеем Super AMOLED с разрешением 1080 x 2400 пикселей, что обеспечивает четкую и яркую картинку. Он также имеет камеру на 64 Мп с возможностью записи видео в 4K. Процессор Snapdragon 732G обеспечивает отличную производительность, а аккумулятор на 5020 мАч обеспечивает долгое время работы.
2. Смартфон Xiaomi Mi 11 Lite: Этот смартфон имеет ультратонкий и легкий дизайн, который удобно держать в руке. Он оснащен 6.55-дюймовым AMOLED-дисплеем с разрешением 1080 x 2400 пикселей. Камера на 64 Мп способна делать качественные фотографии, а быстрый процессор Qualcomm Snapdragon 732G позволяет работать с приложениями плавно.
3. Смартфон Xiaomi Redmi 9T: Этот смартфон оснащен 6.53-дюймовым IPS-дисплеем с разрешением 1080 x 2340 пикселей, который обеспечивает реалистичную цветопередачу. Камера на 48 Мп делает четкие фото, а процессор Snapdragon 662 обеспечивает быструю работу. Аккумулятор на 6000 мАч позволяет использовать устройство долгое время без подзарядки.
4. Смартфон Xiaomi Mi 10T Pro: Этот смартфон оснащен 6.67-дюймовым IPS-дисплеем с разрешением 1080 x 2400 пикселей. Он имеет камеру на 108 Мп, что позволяет делать потрясающие фотографии. Процессор Qualcomm Snapdragon 865 обеспечивает высокую производительность, а аккумулятор на 5000 мАч гарантирует долгое время работы.
5. Смартфон Xiaomi Redmi Note 9 Pro: Этот смартфон оснащен 6.67-дюймовым IPS-дисплеем с разрешением 1080 x 2400 пикселей. Камера на 64 Мп с поддержкой искусственного интеллекта позволяет делать яркие и четкие фотографии. Процессор Snapdragon 720G обеспечивает отличную производительность, а аккумулятор на 5020 мАч обеспечивает долгое время работы.
6. Смартфон Xiaomi Mi 10T Lite: Этот смартфон оснащен 6.67-дюймовым IPS-дисплеем с разрешением 1080 x 2400 пикселей. Он имеет камеру на 64 Мп, что позволяет делать яркие и детальные фотографии. Процессор Qualcomm Snapdragon 750G ускоряет работу с приложениями, а аккумулятор на 4820 мАч обеспечивает долгую автономность.
7. Смартфон Xiaomi Redmi Note 8 Pro: Этот смартфон оснащен 6.53-дюймовым IPS-дисплеем с разрешением 1080 x 2340 пикселей. Камера на 64 Мп делает четкие и яркие фото, а процессор MediaTek Helio G90T обеспечивает плавную работу. Аккумулятор на 4500 мАч достаточно емкий для долгого использования.
8. Смартфон Xiaomi Mi 10 Pro: Этот смартфон оснащен 6.67-дюймовым AMOLED-дисплеем с разрешением 1080 x 2340 пикселей. Камера на 108 Мп и оптическая стабилизация изображения позволяют делать высококачественные фотографии. Процессор Snapdragon 865 ускоряет работу с приложениями, а аккумулятор на 4500 мАч достаточно емкий.
9. Смартфон Xiaomi Redmi 9C: Этот смартфон оснащен 6.53-дюймовым IPS-дисплеем с разрешением 720 x 1600 пикселей. Он имеет камеру на 13 Мп, которая делает четкие фото в хороших условиях освещения. Процессор MediaTek Helio G35 обеспечивает достаточную производительность для базовых задач, а аккумулятор на 5000 мАч обеспечивает долгое время работы.
10. Смартфон Xiaomi Mi 11 Ultra: Этот смартфон оснащен 6.81-дюймовым AMOLED-дисплеем с разрешением 1440 x 3200 пикселей. Он имеет камеру на 50 Мп, а также вспомогательные камеры для различных эффектов съемки. Процессор Snapdragon 888 обеспечивает высокую производительность, а аккумулятор на 5000 мАч гарантирует долгое время работы."""

In [ ]:
class Conversation:
    def __init__(
        self,
        message_template=MESSAGE_TEMPLATE,
        system_prompt=SYSTEM_PROMPT,
        response_template=RESPONSE_TEMPLATE,
        personality=None
    ):
        self.message_template = message_template
        self.response_template = response_template
        if personality:
            system_prompt = system_prompt.format(personality=personality)
        else:
            system_prompt = system_prompt.format(personality="")
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += RESPONSE_TEMPLATE
        return final_text.strip()

In [ ]:
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False)
tokenizer.pad_token_id = 0
generation_config = GenerationConfig(
    pad_token_id=0,
    bos_token_id=1,
    eos_token_id=2,
    temperature=0.2,
    top_p=0.9,
    top_k=40,
    do_sample=True,
    num_beams=1,
    max_new_tokens=1536,
    repetition_penalty=1.1,
    no_repeat_ngram_size=15
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, torch_dtype=torch.float16)
model.eval()

In [ ]:
#personality = "Личность чат-бота: профессиональный и формальный – чат-бот поддерживает официальный, деловой тон."
#conversation = Conversation(personality=personality)
conversation = Conversation()

In [ ]:
while True:
    user_uttr = input("Клиент: ")
    print(f"Клиент: {user_uttr}")
    conversation.add_user_message(user_uttr)
    prompt = conversation.get_prompt(tokenizer)
    output = generate(model, tokenizer, prompt, generation_config)
    output = output.split("bot")[0].replace("<s>", "").replace("</s>", "").strip()
    print(f"Чат-бот: {output}")